# Глубокая нейронная сеть для классификации изображений: на основе pytorch

In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from dnn_app_utils_v3 import load_data
import torch.nn as nn
import numpy as np

## 1. Подготовка данных

In [2]:
train_x_orig, train_y, test_x_orig, test_y, classes = load_data()

train_x_flatten = train_x_orig.reshape(-1, 64 * 64 * 3) 
test_x_flatten = test_x_orig.reshape(-1, 64 * 64 * 3)

train_x = train_x_flatten/255.
test_x = test_x_flatten/255.

train_y = train_y.reshape(-1, 1)
test_y = test_y.reshape(-1, 1)

train_x = train_x.astype(dtype=np.float32)
test_x = test_x.astype(dtype=np.float32)
train_y = train_y.astype(dtype=np.float32)
test_y = test_y.astype(dtype=np.float32)

print ("train_x's shape: " + str(train_x.shape))
print ("test_x's shape: " + str(test_x.shape))
print ("train_y's shape: " + str(train_y.shape))
print ("test_y's shape: " + str(test_y.shape))

train_x's shape: (209, 12288)
test_x's shape: (50, 12288)
train_y's shape: (209, 1)
test_y's shape: (50, 1)


In [3]:
x_train = torch.from_numpy(train_x)
y_train = torch.from_numpy(train_y)

x_test = torch.from_numpy(test_x)
y_test = torch.from_numpy(test_y)

In [4]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

torch.Size([209, 12288]) torch.Size([209, 1]) torch.Size([50, 12288]) torch.Size([50, 1])


In [5]:
train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=50, shuffle=False)

# 2-слойная нейросеть

## 2. Архитектура нейросети 

In [6]:
input_size = 64 * 64 * 3
hidden_size = 7
output_size = 1
num_epochs = 3000
learning_rate = 0.0075

In [7]:
model = nn.Sequential(nn.Linear(input_size, hidden_size), nn.ReLU(), nn.Linear(hidden_size, output_size), nn.Sigmoid()) 
model

Sequential(
  (0): Linear(in_features=12288, out_features=7, bias=True)
  (1): ReLU()
  (2): Linear(in_features=7, out_features=1, bias=True)
  (3): Sigmoid()
)

## 3. Loss & optimizer

In [8]:
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## 4. Обучение

In [9]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 500 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [500/3000], Loss: 0.0284
Epoch [1000/3000], Loss: 0.0103
Epoch [1500/3000], Loss: 0.0050
Epoch [2000/3000], Loss: 0.0039
Epoch [2500/3000], Loss: 0.0010
Epoch [3000/3000], Loss: 0.0010


In [10]:
correct = 0
total = 0
for images, labels in train_loader:
    
    outputs = model(images)
    
    predicted, _ = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted.round() == labels.ravel()).sum()

print('Accuracy of the model on the train images: {} %'.format(100 * correct / total))

Accuracy of the model on the train images: 100.0 %


In [11]:
correct = 0
total = 0
for images, labels in test_loader:
    
    outputs = model(images)
    
    predicted, _ = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted.round() == labels.ravel()).sum()

print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 68.0 %
